<a href="https://colab.research.google.com/github/KazutoYamada/Image_Recognition_for_PoC/blob/main/Image_Recognition_for_PoC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PoC段階での不良品検知用画像認識モデル（Pytorch）

1. **Googleドライブと接続**

In [ ]:
#コラボのマウント
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


2. **ライブラリーのインポート**


In [ ]:
#ライブラリのインポート
import os
import zipfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets,models, transforms

3. **学習データの整理**

In [ ]:
#学習データのzip解凍＆保存
data_dilectory = "/content/drive/My Drive/ManabiDXQuestPBL02PoC/"
files = os.listdir(data_dilectory)
data_file = "02_PBL02_data/" #読み込む学習データのあるファイル名

if data_file in files:
    train_file = data_dilectory + data_file + "train/"
    train_data_file_list = os.listdir(train_file)
    all_data = []
    for train_data in train_data_file_list:
        all_data += os.listdir(train_file + "/" + train_data)
    if (len(all_data)) == len(set(all_data)):
        print("ファイルに重複なし")
    else:
        print("ファイルに重複あり")

elif (data_file + ".zip") in files: #zipファイルなら回答
    with zipfile.ZipFile((data_dilectory + "/" + data_file + ".zip")) as zf:
        zf.extractall(data_dilectory + "/")

else:
    print(data_file + "のファイルがありません。")

test_files = os.listdir("/content/drive/My Drive/ManabiDXQuestPBL02PoC/02_PBL02_data/test/") #213個
test_image = Image.open(data_dilectory + "/" + data_file + "/" + "test/" + test_files[0])


4.**データの前処理**

In [ ]:
#transform作成
data_transforms = {
    'train':transforms.Compose([
        transforms.Resize(256),
        transforms.RandomVerticalFlip(0.5),
        transforms.RandomHorizontalFlip(0.5),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    'val':transforms.Compose([
        transforms.Resize(256),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])}

5.**データの読み込み**

In [ ]:

image_datasets = {
    'train': datasets.ImageFolder('/content/drive/My Drive/ManabiDXQuestPBL02PoC/02_PBL02_data/train', data_transforms['train']),
    'val': datasets.ImageFolder('/content/drive/My Drive/ManabiDXQuestPBL02PoC/02_PBL02_data/val', data_transforms['val']),
    'test' : datasets.ImageFolder('/content/drive/My Drive/ManabiDXQuestPBL02PoC/02_PBL02_data/test', data_transforms['val'])
}


6.**ハイパーパラメータの設定**

In [ ]:
#ハイパーパラメータの設定
batch_size = 16
epochs = 15
lr = 0.001
devise = torch.device("cuda" if torch.cuda.is_available() else "cpu")

7,**事前学習済みモデルの設定**

In [ ]:
resnet_model = models.resnet18(pretrained=True)
resnet_model.fc = nn.Linear(in_features=512, out_features=4, bias=True)
resnet_model = resnet_model.to(devise)
loss_fnc = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet_model.parameters(), lr = lr, momentum=0.9)

In [ ]:
#訓練済みモデルのロードと要素確認と変更
image_dataloaders = {
    'train' : DataLoader(image_datasets['train'], batch_size = batch_size, shuffle= True, num_workers= 0, drop_last = False),
    'val' : DataLoader(image_datasets['val'], batch_size = batch_size, shuffle= False,  num_workers= 0,drop_last = False),
    'test' : DataLoader(image_datasets['test'], batch_size = batch_size, shuffle= False, num_workers= 0, drop_last = False)
}
print(image_datasets['train'].class_to_idx)


{'bridge': 0, 'horn': 1, 'potato': 2, 'regular': 3}


8.**モデルの学習**

In [ ]:
#学習フェーズ
for epoch in range(epochs):
    for phase in ["train", "val"]:
        if phase == "train":
            resnet_model.train()
        else:
            resnet_model.eval()
        total_loss = 0.0
        corrects = 0
        data_size = 0
        for inputs, labels in image_dataloaders[phase]:
            inputs = inputs.to(devise)
            labels = labels.to(devise)

            optimizer.zero_grad()

            outputs = resnet_model(inputs)
            _, pred_labels = torch.max(outputs,1)

            loss = loss_fnc(outputs, labels)

            if phase == "train":
                loss.backward()
                optimizer.step()

            total_loss += loss.item()
            corrects += torch.sum(pred_labels == labels.data)
            data_size += len(labels.data)
        acc = corrects/data_size
        print("{} Loss:{:.4f} Acc:{:.4f}".format(phase, total_loss/data_size, acc))

9.**テストデータの判定**

In [ ]:
#推論フェーズ
resnet_model.eval()

pred_list_4class = []
pred_list = []

for inputs, _ in image_dataloaders['test']:
    inputs = inputs.to(devise)
    outputs = resnet_model(inputs)
    pred_labels = outputs.argmax(1).tolist()
    pred_list_4class.extend(pred_labels)


10.**不良品多クラス分類から良・不良の2値に変更**

In [ ]:
file_list = os.listdir('/content/drive/My Drive/ManabiDXQuestPBL02PoC/02_PBL02_data/test/test/')
for i, label in enumerate(pred_list_4class):
    if label == 3:
        pred_list.append(0)
    else:
        pred_list.append(1)

11.**ファイルの出力**

In [ ]:
#ファイル出力
df = pd.DataFrame([file_list, pred_list]).T
df.to_csv("/content/drive/My Drive/ManabiDXQuestPBL02PoC/my_submission.tsv",
          index = False,
          header = False,
          sep = "\t")